<a href="https://colab.research.google.com/github/anirudh14397/Stock-Market-Analysis/blob/main/Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stock Market Performance Analysis**

Stock market performance analysis can be used to inform investment decisions and help investors make informed decisions about buying or selling stocks. 

Importing the necessary Python libraries and the dataset. For this task, I will use the Yahoo finance API (yfinance) to collect real-time stock market data for the past three months.

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime

we first imported the necessary Python libraries and downloaded the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.

In [3]:
start_date=datetime.now() - pd.DateOffset(months=3)
end_date=datetime.now()
tickers = ['AAPL','MSFT','NFLX','GOOG']
df_list=[]
for ticker in tickers:
  data = yf.download(ticker,start=start_date,end=end_date)
  df_list.append(data)

df=pd.concat(df_list,keys=tickers,names=['Ticker','Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-13  147.809998  153.139999  147.699997  150.470001  150.262161   
       2023-03-14  151.279999  153.399994  150.100006  152.589996  152.379227   
       2023-03-15  151.190002  153.250000  149.919998  152.990005  152.778687   
       2023-03-16  152.160004  156.460007  151.639999  155.850006  155.634735   
       2023-03-17  156.080002  156.740005  154.279999  155.000000  154.785904   

                     Volume  
Ticker Date                  
AAPL   2023-03-13  84457100  
       2023-03-14  73695900  
       2023-03-15  77167900  
      

The Date column is the index column in the DataFrame. We need to reset the index 

In [4]:
df=df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-13  147.809998  153.139999  147.699997  150.470001   
1   AAPL 2023-03-14  151.279999  153.399994  150.100006  152.589996   
2   AAPL 2023-03-15  151.190002  153.250000  149.919998  152.990005   
3   AAPL 2023-03-16  152.160004  156.460007  151.639999  155.850006   
4   AAPL 2023-03-17  156.080002  156.740005  154.279999  155.000000   

    Adj Close    Volume  
0  150.262161  84457100  
1  152.379227  73695900  
2  152.778687  77167900  
3  155.634735  76161100  
4  154.785904  98944600  


Performance in the stock market of all the companies

In [6]:
import plotly.express as px
fig=px.line(df,x='Date',y='Close',color='Ticker',title='Stock Market Performance for Last 3 months')
fig.show()

faceted area chart

In [7]:
fig=px.area(df,x='Date',y='Close',color='Ticker',facet_col='Ticker',labels={'Date':'Date','Close':'Closing Price','Ticker':'Company'},title='Stock Prices for Apple,Microsoft,Netflix and Google')
fig.show()

Moving averages, which provide a useful way to identify trends and patterns in each company’s stock price movements over a period of time

In [8]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  176.118999  174.669001
59  176.620000  174.954501
60  177.246001  175.257001
61  178.119002  175.607502
62  178.916002  175.968002

[63 rows x 2 columns]
Moving Averages for GOOG
           MA10      MA20
189         NaN       NaN
190         NaN       NaN
191         NaN       NaN
192         NaN       NaN
193         NaN       NaN
..          ...       ...
247  124.482001  120.6700
248  124.686001  121.6535
249  124.651001  122.4035
250  124.754001  122.9230
251  124.606001  123.2215

[63 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  327.262003  319.585999
122  328.512003  32

Moving averages of all companies

In [9]:
for ticker,group in df.groupby('Ticker'):
  fig=px.line(group,x='Date',y=['Close','MA10','MA20'],title=f"{ticker} Moving Averages")

  fig.show()

When the MA10 crosses above the MA20, it is considered a bullish signal indicating that the stock price will continue to rise. Conversely, when the MA10 crosses below the MA20, it is a bearish signal that the stock price will continue falling.

Volatility is a measure of how much and how often the stock price or market fluctuates over a given period of time. 

In [10]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

Let’s analyze the correlation between the stock prices of Apple and Microsoft:

In [11]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()

There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase. It is a sign of a strong correlation or similarity between the two companies, which can be due to factors such as industry trends, market conditions, or common business partners or customers. For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.